In [4]:
import pandas as pd
from gurobipy import Model, GRB
import gurobipy as gp
from gurobipy import GRB

In [2]:
df = pd.read_csv('/Users/pratiksha/Downloads/BasketballPlayers.csv')

In [3]:
df.head()

,Number,Position,Ball Handling,Shooting,Rebounding,Defense,Athletic Ability,Toughness,Mental Acuity
0,1,G/F,1,2,3,2,1,2,1
1,2,G/F,1,1,1,2,3,2,3
2,3,G/F,3,1,1,2,3,2,1
3,4,G/F,2,3,2,2,2,1,1
4,5,F/C,1,2,3,3,3,3,2


In [9]:
df['Average'] = df.iloc[:, 2:].mean(axis=1)

In [10]:
# Filter players with average skill above 2.05 (without resetting the index)
filtered_players_df = df[df['Average'] > 2.05]
filtered_players_df = filtered_players_df.drop(columns=['Average'])

In [11]:
# Number of players
num_players = len(filtered_players_df)

In [12]:
# Create the optimization model
model = Model("TrainingCampSelection")

In [13]:
# Create binary decision variables for each player (using original indices)
x = model.addVars(filtered_players_df.index, vtype=GRB.BINARY, name="Player")

In [14]:
# Pre-compute the player positions using original indices
guards = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['G', 'G/F']]
forwards_centers = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['F', 'C', 'F/C']]

In [15]:
# Total number of players selected
total_players_selected = sum(x[i] for i in filtered_players_df.index)

In [16]:
# At least 30% of the invitations should go to guards
model.addConstr(sum(x[i] for i in guards) >= 0.3 * total_players_selected, "Min_30_percent_guards")

<gurobi.Constr *Awaiting Model Update*>

In [17]:
# At least 40% of the invitations should go to forwards/centers
model.addConstr(sum(x[i] for i in forwards_centers) >= 0.4 * total_players_selected, "Min_40_percent_forwards_centers")

<gurobi.Constr *Awaiting Model Update*>

In [18]:
# Limit the total number of invitations to 21
model.addConstr(total_players_selected <= 21, "Total_Invitations_Limit")

<gurobi.Constr *Awaiting Model Update*>

In [19]:
# If any player from 20-24 (inclusive) is invited, all players from 72-78 (inclusive) cannot be, and vice versa
model.addConstr(sum(x[i] for i in filtered_players_df.index if 20 <= filtered_players_df.loc[i, 'Number'] <= 24) + sum(x[j] for j in filtered_players_df.index if 72 <= filtered_players_df.loc[j, 'Number'] <= 78) <= 1, "Group_20_24_vs_72_78")

<gurobi.Constr *Awaiting Model Update*>

In [20]:
# If any player from 105-114 (inclusive) is invited, at least one player from 45-49 (inclusive) and 65-69 (inclusive) must be invited
for i in [idx for idx in filtered_players_df.index if 105 <= filtered_players_df.loc[idx, 'Number'] <= 114]:
    model.addConstr(x[i] <= sum(x[j] for j in filtered_players_df.index if 45 <= filtered_players_df.loc[j, 'Number'] <= 49), f"Group_105_114_requires_45_49_for_{i}")
    model.addConstr(x[i] <= sum(x[k] for k in filtered_players_df.index if 65 <= filtered_players_df.loc[k, 'Number'] <= 69), f"Group_105_114_requires_65_69_for_{i}")


In [21]:
# At least one player must be invited from: 1-10, 11-20, 21-30, ..., 131-140, 141-150
for i in range(1, 151, 10):
    model.addConstr(sum(x[j] for j in filtered_players_df.index if i <= filtered_players_df.loc[j, 'Number'] < i + 10) >= 1, f"Group_{i}_{i+9}")

In [22]:
# Objective function to maximize total skill ratings
skills = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']
model.setObjective(sum(filtered_players_df.loc[i, skill] * x[i] for i in filtered_players_df.index for skill in skills), GRB.MAXIMIZE)

In [23]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 29 rows, 65 columns and 295 nonzeros
Model fingerprint: 0xa42ff278
Variable types: 0 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 336.0000000
Presolve removed 1 rows and 12 columns
Presolve time: 0.01s
Presolved: 28 rows, 53 columns, 246 nonzeros
Variable types: 0 continuous, 53 integer (46 binary)
Found heuristic solution: objective 353.0000000

Root relaxation: objective 3.600000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0           

In [24]:
# Print the selected players and their details
selected_players = [i for i in filtered_players_df.index if x[i].X == 1]
count_guards = sum(1 for i in selected_players if i in guards)
count_forwards_centers = sum(1 for i in selected_players if i in forwards_centers)
total_selected = len(selected_players)
num_decision_variables = len(filtered_players_df)

In [25]:
print(f"Number of decision variables: {num_decision_variables}")

Number of decision variables: 65


In [26]:
print("Selected players and their positions:")
for i in selected_players:
    print(f"Player {i}: {filtered_players_df.loc[i, 'Position']}")
print(f"Total guards selected: {count_guards}")
print(f"Total forwards/centers selected: {count_forwards_centers}")
print(f"Total players selected: {total_selected}")

Selected players and their positions:
Player 4: F/C
Player 6: F/C
Player 10: F
Player 25: G
Player 36: G/F
Player 46: G/F
Player 55: F/C
Player 66: G
Player 73: F/C
Player 89: F
Player 94: G/F
Player 103: G
Player 104: F/C
Player 109: G
Player 110: F
Player 117: G/F
Player 127: G/F
Player 131: G/F
Player 132: F
Player 133: F/C
Player 143: F
Total guards selected: 10
Total forwards/centers selected: 11
Total players selected: 21


In [27]:
# Print the value of the objective function
print(f"Value of objective function: {model.ObjVal}")

Value of objective function: 360.0
